In [1]:
from pathlib import Path
import os
import pandas as pd
import re
import numpy as np

In [2]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

In [3]:
from transformers import BertTokenizerFast, BertModel, AdamW, get_linear_schedule_with_warmup

In [4]:
from project_path import project_path
project_path(1)

d:\Users\Nicholas\Projects\NLP\fake_news


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve, auc, precision_recall_fscore_support

In [6]:
# import from src
import src.models.models as models
import src.models.data_loader as data_loader
import src.models.training as training

In [7]:
import pickle

In [8]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
# seeds
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [10]:
# if gpu is available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [11]:
# data directories
proj_dir = Path.cwd().parents[0]
data_raw = Path(proj_dir, 'data', 'raw')

In [12]:
data_fake = pd.read_csv(Path(data_raw, 'Fake.csv'))
data_true = pd.read_csv(Path(data_raw, 'True.csv'))

In [13]:
data_fake['target'] = 1
data_true['target'] = 0

In [14]:
data = pd.concat([data_fake, data_true])
data_sample, _ = train_test_split(data, train_size=0.1, random_state=RANDOM_SEED, shuffle=True, stratify=data['target'])
data_sample = data_sample.rename(columns = {'text':'content'})

#### Train test split

In [15]:
df_train, df_test = train_test_split(
    data_sample, 
    train_size=0.8, 
    random_state=RANDOM_SEED, 
    shuffle=True,
    stratify=data_sample['target'])

In [16]:
df_val, df_test = train_test_split(
    df_test, 
    train_size=0.5, 
    random_state=RANDOM_SEED, 
    shuffle=True, 
    stratify=df_test['target'])

In [17]:
print(df_train.shape, df_val.shape, df_test.shape)

(3591, 5) (449, 5) (449, 5)


In [18]:
print(df_train.target.value_counts(normalize = True))

1    0.522974
0    0.477026
Name: target, dtype: float64


#### Train model

In [19]:
PRE_TRAINED_MODEL_NAME = "D:/Users/Nicholas/Projects/BERT_pretrained/biobert-base-cased-v1.1"
tokenizer = BertTokenizerFast.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [20]:
max_len = 512 * 3
batch_size = 32

In [21]:
train_data_loader = data_loader.create_data_loader(
    df = df_train, 
    tokenizer = tokenizer, 
    max_len = max_len, 
    batch_size = batch_size, 
    chunksize = 512, 
    sampler = None, 
    shuffle = True, 
    drop_last = True)

val_data_loader = data_loader.create_data_loader(
    df = df_val, 
    tokenizer = tokenizer, 
    max_len = max_len, 
    batch_size = batch_size, 
    chunksize = 512, 
    sampler = None, 
    shuffle = False, 
    drop_last = False)

test_data_loader = data_loader.create_data_loader(
    df = df_test, 
    tokenizer = tokenizer, 
    max_len = max_len, 
    batch_size = batch_size, 
    chunksize = 512, 
    sampler = None, 
    shuffle = False, 
    drop_last = False)

In [22]:
params_dict = {
    'PRE_TRAINED_MODEL_NAME':PRE_TRAINED_MODEL_NAME, 
    'n_classes':2, 
    'add_linear':[512,256], 
    'attn_bias':False, 
    'freeze_layer_count':8
}

In [23]:
model = models.HIBERT(**params_dict)

Some weights of the model checkpoint at D:/Users/Nicholas/Projects/BERT_pretrained/biobert-base-cased-v1.1 were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)

bert.embeddings.word_embeddings.weight False
bert.embeddings.position_embeddings.weight False
bert.embeddings.token_type_embeddings.weight False
bert.embeddings.LayerNorm.weight False
bert.embeddings.LayerNorm.bias False
bert.encoder.layer.0.attention.self.query.weight False
bert.encoder.layer.0.attention.self.query.bias False
bert.encoder.layer.0.attention.self.key.weight False
bert.encoder.layer.0.attention.self.key.bias False
bert.encoder.layer.0.attention.self.value.weight False
bert.encoder.layer.0.attention.self.value.bias False
bert.encoder.layer.0.attention.output.dense.weight False
bert.encoder.layer.0.attention.output.dense.bias False
bert.encoder.layer.0.attention.output.LayerNorm.weight False
bert.encoder.layer.0.attention.output.LayerNorm.bias False
bert.encoder.layer.0.intermediate.dense.weight False
bert.encoder.layer.0.intermediate.dense.bias False
bert.encoder.layer.0.output.dense.weight False
bert.encoder.layer.0.output.dense.bias False
bert.encoder.layer.0.output.Lay

In [25]:
epochs = 2
optimizer = AdamW(model.parameters(), lr=1e-5, correct_bias=False)

total_steps = len(train_data_loader) * epochs

scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)

In [26]:
%%time
history = training.train_model(epochs, model, train_data_loader, val_data_loader, loss_fn, optimizer, device, scheduler)

Epoch 1 / 2
--------------------
Batch Train loss: 0.6941     Class 1 prop: 0.4688     ACC: 0.5938     F1: 0.6667     AUC: 0.5139     
[1 0 0 1 0 0 0 0 1 1 0 1 1 1 1 0 0 1 1 1 1 0 0 1 0 0 0 0 1 0 1 0] [0.5120627  0.50814855 0.5102492  0.50960124 0.50968945 0.5126172
 0.5120235  0.5110454  0.51118624 0.50971764 0.5135421  0.51240885
 0.5129199  0.50987375 0.51148874 0.5100291  0.50921625 0.50675637
 0.5121607  0.5127731  0.5106375  0.50939375 0.5121539  0.5080478
 0.506956   0.51340556 0.51161134 0.5080251  0.5131007  0.5076514
 0.512083   0.5103428 ]

Epoch Train loss: 0.6941     ACC: 0.5938     F1: 0.6667     AUC: 0.5139     
Epoch val  loss: 0.6946     ACC: 0.5000     F1: 0.0000     AUC: 0.2917     

Epoch 2 / 2
--------------------
Batch Train loss: 0.6919     Class 1 prop: 0.4688     ACC: 0.7812     F1: 0.7879     AUC: 0.7479     
[1 1 1 1 0 1 0 0 1 1 1 0 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1] [0.50779    0.506229   0.5036712  0.5062225  0.50620043 0.50652605
 0.5022381  0.504586

In [27]:
pickle.dump(history, open('training_history.pkl', 'wb'))

#### plot history

In [28]:
history = pickle.load(history, open('training_history.pkl', 'rb'))

TypeError: load() takes exactly 1 positional argument (2 given)

In [29]:
plt.plot(list(range(epochs)), history['train_loss'], label = 'train loss')
plt.plot(list(range(epochs)), history['val_loss'], label = 'val loss')
plt.legend()

TypeError: 'NoneType' object is not subscriptable

In [30]:
plt.plot(list(range(epochs)), history['train_f1'], label = 'train f1')
plt.plot(list(range(epochs)), history['val_f1'], label = 'val f1')
plt.legend()

TypeError: 'NoneType' object is not subscriptable

In [31]:
plt.plot(list(range(epochs)), history['train_auc'], label = 'train auc')
plt.plot(list(range(epochs)), history['val_auc'], label = 'val auc')
plt.legend()

TypeError: 'NoneType' object is not subscriptable

#### make predictions

In [32]:
model_trained = models.HIBERT(**params_dict)
model_trained.load_state_dict(torch.load('best_model_state.bin'))

best_threshold = pickle.load(history, open('best_threshold.pkl', 'rb'))

Some weights of the model checkpoint at D:/Users/Nicholas/Projects/BERT_pretrained/biobert-base-cased-v1.1 were not used when initializing BertModel: ['bert.pooler.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'bert.pooler.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


FileNotFoundError: [Errno 2] No such file or directory: 'best_model_state.bin'

In [33]:
predictions = training.pred_model(
    model = model_trained, 
    data_loader = test_data_loader, 
    device = device, 
    best_threshold = best_threshold)

NameError: name 'best_threshold' is not defined